In [ ]:
! pip install transformers

     |████████████████████████████████| 2.3MB 4.3MB/s 
     |████████████████████████████████| 3.3MB 19.6MB/s 
     |████████████████████████████████| 901kB 37.1MB/s 


In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization")
ARTICLE = """  usain bolt rounded off the world championships sunday by claiming his third gold in moscow as he anchored jamaica to victory in the men is 4x100m relay. the fastest man in the world charged clear of 
united states rival justin gatlin as the jamaican quartet of nesta carter, kemar bailey cole, nickel ashmeade and bolt won in 37.36 seconds. the u.s finished second in 37.56 seconds with canada taking the bronze after 
britain were disqualified for a faulty handover. the 26 year old bolt has now collected eight gold medals at world championships, equaling the record held by american trio carl lewis, michael johnson and allyson felix,
 not to mention the small matter of six olympic titles. the relay triumph followed individual successes in the 100 and 200 meters in the russian capital. i am proud of myself and i will continue to work to dominate for
  as long as possible, bolt said, having previously expressed his intention to carry on until the 2016 rio olympics. victory was never seriously in doubt once he got the baton safely in hand from ashmeade, while gatlin 
  and the united states third leg runner rakieem salaam had problems. gatlin strayed out of his lane as he struggled to get full control of their baton and was never able to get on terms with bolt. earlier, jamaica is 
  women underlined their dominance in the sprint events by winning the 4x100m relay gold, anchored by shelly ann fraser pryce, who like bolt was completing a triple. their quartet recorded a championship record of
   41.29 seconds, well clear of france, who crossed the line in second place in 42.73 seconds. defending champions, the united states, were initially back in the bronze medal position after losing time on the second 
   handover between alexandria anderson and english gardner, but promoted to silver when france were subsequently disqualified for an illegal handover. the british quartet, who were initially fourth, were promoted to 
   the bronze which eluded their men is team. fraser pryce, like bolt aged 26, became the first woman to achieve three golds in the 100 200 and the relay. in other final action on the last day of the championships, 
   france is teddy tamgho became the third man to leap over 18m in the triple jump, exceeding the mark by four centimeters to take gold. germany is christina obergfoll finally took gold at global level in the women 
   is javelin after five previous silvers, while kenya is asbel kiprop easily won a tactical men is 1500m final. kiprop is compatriot eunice jepkoech sum was a surprise winner of the women is 800m. bolt is final dash 
   for golden glory brought the eight day championship to a rousing finale, but while the hosts topped the medal table from the united states there was criticism of the poor attendances in the luzhniki stadium. there 
   was further concern when their pole vault gold medalist yelena isinbayeva made controversial remarks in support of russia is new laws, which make the propagandizing of non traditional sexual relations among minors 
   a criminal offense. she later attempted to clarify her comments, but there were renewed calls 
by gay rights groups for a boycott of the 2014 winter games in sochi, the next major sports event in russia. 
"""

In [ ]:
len(ARTICLE)

3241

In [ ]:
result_t=(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))
len(result_t)

1

In [ ]:
print(result_t[0].get('summary_text'))
len(result_t[0].get('summary_text'))
result_sum = result_t[0].get('summary_text')


 usain bolt rounded off the world championships sunday by claiming his third gold in the men is 4x100m relay . The jamaican quartet of nesta carter, kemar bailey cole, nickel ashmeade and bolt won in 37.36 seconds . The 26 year old bolt has now collected eight gold medals at world championships, equaling the record held by carl lewis, michael johnson and allyson felix .


In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer
model = AutoModelWithLMHead.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")
# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer.encode("summarize: " + ARTICLE, return_tensors="pt", max_length=512)
outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:810: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
result_t5=tokenizer.decode(outputs[0])
type(result_t5)
print(result_t5)
result_sum_t5='usain bolt wins third gold at world championships in moscow. anchors jamaica to victory in men is 4x100m relay. u.s finish second in 37.36 seconds. canada take bronze after disqualifying britain for faulty handover'
print(result_sum_t5)

<pad> usain bolt wins third gold at world championships in moscow. anchors jamaica to victory in men is 4x100m relay. u.s finish second in 37.36 seconds. canada take bronze after disqualifying britain for faulty handover.</s>
usain bolt wins third gold at world championships in moscow. anchors jamaica to victory in men is 4x100m relay. u.s finish second in 37.36 seconds. canada take bronze after disqualifying britain for faulty handover


In [ ]:
!pip install sumeval
!pip install spacy

     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 61kB 4.8MB/s 
  Created wheel for sumeval: filename=sumeval-0.2.2-cp37-none-any.whl size=54536 sha256=92afa0a87e82540cddd088c93d4da6efbd54498c40e2efad8289377aacc07c8d
  Stored in directory: /root/.cache/pip/wheels/7b/6f/57/19ceecab21445c88f3c565735fa1887b4cd18d340c972eb445
Successfully built sumeval


In [ ]:
from sumeval.metrics.rouge import RougeCalculator
from sumeval.metrics.bleu import BLEUCalculator

def eval_rouges(refrence_summary,model_summary):
    rouge = RougeCalculator(stopwords=True, lang="en")

    rouge_1 = rouge.rouge_n(
                summary=model_summary,
                references=refrence_summary,
                n=1)
    print("rogue1 "+str(rouge_1))

    rouge_2 = rouge.rouge_n(
                summary=model_summary,
                references=[refrence_summary],
                n=2)
    print("rogue2 "+str(rouge_2))

    rouge_l = rouge.rouge_l(
                summary=model_summary,
                references=[refrence_summary])
    print("roguel "+str(rouge_l))

    
    # You need spaCy to calculate ROUGE-BE
    
    #rouge_be = rouge.rouge_be(
    #            summary=model_summary,
    #            references=[refrence_summary])
    #print("rogue_be "+str(rouge_be))


    bleu = BLEUCalculator()
    bleu_score = bleu.bleu( summary=model_summary,
                        references=[refrence_summary])
    
    print("blue_score "+str(bleu_score))
    
    return rouge_1, rouge_2,rouge_l,bleu_score

In [ ]:
ref_sum='usain bolt wins third gold of world championship anchors jamaica to 4x100m relay victory eighth gold at the championships for bolt jamaica double up in women is 4x100m relay'
eval_rouges(ref_sum,result_sum)

rogue1 0.29032258064516125
rogue2 0.06666666666666667
roguel 0.22580645161290322
blue_score 1.3040481494516774


(0.29032258064516125,
 0.06666666666666667,
 0.22580645161290322,
 1.3040481494516774)

In [ ]:
eval_rouges(ref_sum,result_sum_t5)

rogue1 0.5116279069767442
rogue2 0.2926829268292683
roguel 0.46511627906976744
blue_score 16.685328082754545


(0.5116279069767442,
 0.2926829268292683,
 0.46511627906976744,
 16.685328082754545)